# Centralized Learning Simulator

Simulating `Centralized Learning` paradigm.

In [ ]:
import import_ipynb
import nn.ml as ml
import nn.nets as nets

In [ ]:
if __name__ == "__main__":
    import os
    from copy import deepcopy

    import torch
    import torch.nn as nn
    import torch.optim as optim
    from torch.autograd import Variable

    import torchvision.datasets as dset
    import torchvision.transforms as transforms

    from torch.utils.data import DataLoader  # TODO: DistributedDataParallel

    """Hyperparams"""
    numWorkers = 4
    cuda = True

    base_path = './simul_centralized'
    os.makedirs(base_path, exist_ok=True)
    trainFile = open(os.path.join(base_path, 'train.csv'), 'w')
    testFile = open(os.path.join(base_path, 'test.csv'), 'w')

    epochs = 3000 #3000
    batchSz = 64

    """Datasets"""
    # # gets mean and std
    # transform = transforms.Compose([transforms.ToTensor()])
    # dataset = dset.CIFAR10(root='cifar', train=True, download=True, transform=transform)
    # normMean, normStd = dist.get_norm(dataset)
    #normMean = [0.49139968, 0.48215841, 0.44653091]
    normMean = [0]
    #normStd = [0.24703223, 0.24348513, 0.26158784]
    normStd = [1]
    normTransform = transforms.Normalize(normMean, normStd)

    trainTransform = transforms.Compose([
        transforms.RandomCrop(28, padding=4), #CIFAR10은 28 아니고 32
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normTransform
    ])
    testTransform = transforms.Compose([
        transforms.ToTensor(),
        normTransform
    ])

    trainset = dset.MNIST(root='MNIST', train=True, download=True, transform=trainTransform)
    testset = dset.MNIST(root='MNIST', train=False, download=True, transform=trainTransform)

    # num_workers: number of CPU cores to use for data loading
    # pin_memory: being able to speed up the host to device transfer by enabling
    kwargs = {'num_workers': numWorkers, 'pin_memory': cuda}

    # loaders
    trainLoader = DataLoader(trainset, batch_size=batchSz, shuffle=True, **kwargs)
    testLoader = DataLoader(testset, batch_size=batchSz, shuffle=True, **kwargs)

    """Nets"""
    num_classes = 5
    # resnet = nets.resnet18(num_classes=num_classes)
    fcnn = nets.FCNN()

    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(fcnn.parameters(), lr=1e-1, momentum=0.9)

    if cuda:
        # if multi-gpus
        if torch.cuda.device_count() > 1:
            fcnn = nn.DataParallel(fcnn)

        # use cuda
        fcnn.cuda()

    """Train & Test models"""
    for epoch in range(epochs):
        ml.train(
            fcnn, criterion, optimizer, trainLoader,
            epoch=epoch, cuda=cuda, log=True, log_file=trainFile
            # alpha=0.9, temperature=4
        )
        ml.test(
            fcnn, criterion, testLoader,
            epoch=epoch, cuda=cuda, log=True, log_file=testFile
        )